<a href="https://colab.research.google.com/github/Soutrik05/Cobebase_search_using_RAG/blob/main/codebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dependencies installation


In [ ]:
pip install transformers langchain faiss-cpu sentence_transformers tf-keras tensorflow gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
!pip install sentence-transformers

In [ ]:
import os
import mimetypes
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Function to convert files to .txt
def convert_files_to_txt(src_dir, dst_dir):
    """Convert text-compatible files from the source directory to .txt while preserving structure."""

    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            file_path = os.path.join(root, file)

            # Skip binary files by MIME type
            mime_type, _ = mimetypes.guess_type(file_path)
            if mime_type and (mime_type.startswith('image') or mime_type.startswith('audio') or mime_type.startswith('video')):
                continue

            rel_path = os.path.relpath(file_path, src_dir)
            new_root = os.path.join(dst_dir, os.path.dirname(rel_path))
            os.makedirs(new_root, exist_ok=True)

            try:
                if file.endswith('.ipynb'):
                    # Handle Jupyter notebook files
                    with open(file_path, 'r', encoding='utf-8') as f:
                        notebook_data = json.load(f)
                    content = ""
                    for cell in notebook_data.get('cells', []):
                        if cell.get('cell_type') in ['code', 'markdown']:
                            content += "\n".join(cell.get('source', [])) + "\n"
                else:
                    # Handle other text-based files
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()

                # Write the content to a new .txt file
                new_file_path = os.path.join(new_root, os.path.splitext(file)[0] + '.txt')
                with open(new_file_path, 'w', encoding='utf-8') as f:
                    f.write(content)

            except (UnicodeDecodeError, json.JSONDecodeError, OSError) as e:
                print(f"Could not process {file_path}: {e}")

# Clone the repository
os.system("git clone https://github.com/Soutrik05/juris_ai.git")

# Define source and destination directories
src_directory = 'juris_ai'
dst_directory = 'juris_ai_txt'

# Convert files to text
convert_files_to_txt(src_directory, dst_directory)

print(f"Files from '{src_directory}' have been converted and saved in '{dst_directory}'.")


Could not process juris_ai/legal_index.faiss: 'utf-8' codec can't decode byte 0x80 in position 4: invalid start byte
Could not process juris_ai/.git/index: 'utf-8' codec can't decode byte 0xa4 in position 13: invalid start byte
Could not process juris_ai/.git/objects/pack/pack-7e79ca8d9f983c1b0b828c2f6b3699d17528ed6b.idx: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Could not process juris_ai/.git/objects/pack/pack-7e79ca8d9f983c1b0b828c2f6b3699d17528ed6b.pack: 'utf-8' codec can't decode byte 0x9a in position 12: invalid start byte
Files from 'juris_ai' have been converted and saved in 'juris_ai_txt'.


Split Text into Chunks

In [ ]:
def read_corpus(directory):
    corpus = ""
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    corpus += f.read() + "\n"
    return corpus

corpus_text = read_corpus(dst_directory)

# Save merged corpus
with open("legal_corpus.txt", "w", encoding="utf-8") as f:
    f.write(corpus_text)

print("Corpus has been saved as 'legal_corpus.txt'.")

Corpus has been saved as 'legal_corpus.txt'.


In [ ]:
def split_into_chunks(text, chunk_size=1500, overlap=150):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_into_chunks(corpus_text)

# Save chunks to a file
with open("chunks.txt", "w", encoding="utf-8") as f:
    for chunk in chunks:
        f.write(chunk + "\n")

print("Text has been split into chunks and saved.")


Text has been split into chunks and saved.


Create a FAISS Index

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def create_embeddings(text_chunks):
    embeddings = embedding_model.encode(text_chunks)
    return embeddings

embeddings = create_embeddings(chunks)

# Create a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Save FAISS index
faiss.write_index(index, "index.faiss")

print("FAISS index created and saved.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS index created and saved.


Retrieve Relevant Chunks

In [ ]:
import numpy as np

def retrieve_top_chunks(query, index, chunks, k=2):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    results = [chunks[i] for i in indices[0]]
    return results

# Load the FAISS index
index = faiss.read_index("index.faiss")

# Test retrieval
query = "How does the process_query function work in this code?"
top_chunks = retrieve_top_chunks(query, index, chunks)
print("\n".join(top_chunks))


ne, None
    index = faiss.read_index(faiss_file)
    with open(chunks_file, "r", encoding="utf-8") as f:
        chunks = f.read().splitlines()
    return True, index, chunks

def retrieve_top_chunks(query, index, chunks, k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    results = [chunks[i] for i in indices[0]]
    return results

def generate_answer(query, context):
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    response = generator(prompt, max_length=200, num_return_sequences=1)
    return response[0]["generated_text"]

def process_query(query, index, chunks):
    try:
        # Retrieve top-k chunks
        top_chunks = retrieve_top_chunks(query, index, chunks, k=5)
        context = "\n\n".join(top_chunks)
        # Generate answer
        answer = generate_answer(query, context)
        return answer,top_chunks
    except Exception as e:
        return "Error processing quer

Text Generation Using google/flan


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def generate_answer(query, context):
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    response = generator(prompt, max_length=200, num_return_sequences=1)
    return response[0]["generated_text"]

# Combine retrieval and generation
def retrieve_and_generate_answer(query, index, chunks, k=5):
    top_chunks = retrieve_top_chunks(query, index, chunks, k)
    context = "\n\n".join(top_chunks)
    answer = generate_answer(query, context)
    return answer, top_chunks

Device set to use cuda:0


Combine Retrieval and Generation

In [ ]:
import gradio as gr

# Function to process the query
def process_query(query):
    try:
        answer, context_chunks = retrieve_and_generate_answer(query, index, chunks)
        return answer, "\n".join(context_chunks)
    except Exception as e:
        return "Error processing query.", str(e)

# Welcome page with chat transition
def welcome_page():
    with gr.Blocks() as demo:
        demo.css = """
            .container {
                background-color: #6a4c93;
                color: white;
                font-family: 'Arial', sans-serif;
            }
            .title {
                font-size: 36px;
                font-weight: bold;
                color: white;
            }
            .subtitle {
                font-size: 18px;
                margin-top: 10px;
                color: #e0e0e0;
            }
            .gr-button {
                background-color: #8e4e99;
                color: white;
                border: none;
                font-size: 16px;
                padding: 10px 20px;
                border-radius: 5px;
            }
            .gr-button:hover {
                background-color: #7a3b84;
            }
        """

        # Title and Subtitle
        gr.Markdown('<div class="title">Welcome to **CODEBASE QnA**</div>', elem_id="title")
        gr.Markdown('<div class="subtitle">A codebase search Q&A system powered by AI. Click below to start.</div>', elem_id="subtitle")

        # State to control visibility
        chat_visible = gr.State(False)

        # Start Chat Button
        btn_start = gr.Button("Start Chat", elem_id="start-btn")

        # Container for chatbot (initially hidden)
        with gr.Column(visible=False) as chatbot_interface:
            gr.Markdown("### Legal Q&A Chatbot")
            user_input = gr.Textbox(label="Enter your the question you might want to ask about the code")
            answer_output = gr.Textbox(label="Answer")
            context_output = gr.Textbox(label="Context Chunks")

            user_input.submit(process_query, inputs=user_input, outputs=[answer_output, context_output])

        # Function to show chatbot when button clicked
        def show_chat():
            return gr.update(visible=True)

        btn_start.click(show_chat, outputs=chatbot_interface)

    return demo

# Run the App
app = welcome_page()
app.launch(debug=True)



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d0a4edfc1b4a05256c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Token indices sequence length is longer than the specified maximum sequence length for this model (2697 > 512). Running this sequence through the model will result in indexing errors


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d0a4edfc1b4a05256c.gradio.live
